*__Note:__* Filepaths should be changed as necessary

In [5]:
import pandas as pd
import os
import numpy as np
from itertools import compress
import re

In [2]:
file_directory = "/home/NIDDK/DATA/TEDDY/"
file_list = os.listdir(file_directory)
#filter for CSV files
file_list_mask = [".csv" in file.lower() for file in file_list]
file_list = list(compress(file_list, file_list_mask))
#remove the '.csv'
file_list = [re.sub('\.csv', '', file_name, flags = re.IGNORECASE) for file_name in file_list]
file_list_match_regex = '(' + '|'.join(file_list) + ")"


def file_match(value, regex):
    match_object = re.match(regex, value)
    if match_object == None:
        return None
    else:
        return match_object[1]
    
def find_original_column(value, file_regex):
    match_object = re.search(file_regex + "(_binned_|.*bin_[0-3]_)?" + "(.*)", value)
    if match_object == None:
        return None
    else:
        return match_object[3]

def create_data_dictionary(data_file, bin_file, imputed_file, file_regex):
    """
    Objective: Create data dictionary describing AI ready dataset
    
    Parameters:
        - data_file (pd.DataFrame): Input dataframe of AI-ready dataset
        - bin_file (pd.DataFrame): Input dataframe of timepoint bins
        - imputed_file (pd.DataFrame): Input dataframe of columns that were imputed
        - file_regex (list): List of TEDDY filenames
 
    Returns:
        - pd.DataFrame: A dataframe of a human-readable data dictionary for the AI ready dataset
    """
        
    df = pd.read_csv(data_file, low_memory = False)
    is_imputed = pd.read_csv(imputed_file)
    bin_df = pd.read_csv(bin_file, low_memory = False)
    
    data_dict_df = pd.DataFrame()
    
    #Create columns for data dictionary file
    #Column 1
    data_dict_df['feature name'] = df.columns
    
    #Column 2 & 3
    dtype_list = []
    range_list = []
    source_list = []
    

    for col in df.columns:
        #Data type 
        dtype_list.append(df[col].dtype)
        
        #Range of values
        try:
            column_min = df[col].min()
            column_max = df[col].max()
            range_list.append(f"{column_min:.2f} - {column_max:.2f}")
        except: range_list.append(np.nan)

    #Column 4
    #Obtain filename + column name
    for i in df.columns.str.split('_'):
        if len(i) < 3:
            source_list.append(i[0])
        elif len(i) == 3:
            source_list.append(f"{i[0]} {i[1]} + {i[2]}")
        elif len(i) > 3:
            source_list.append(f"{i[0]} {i[1]} + {i[2]}")
    
    #COlumn 6
    imputed_cols = []
    not_imputed_cols = []
    for col in is_imputed.columns:
        if any(is_imputed[col].astype(str).str.contains("Imputed")):
            imputed_cols.append(col)
        if not any(is_imputed[col].astype(str).str.contains("Imputed")):
            not_imputed_cols.append(col)
   
    
    data_dict_df['data type'] = dtype_list
    data_dict_df['values (range of the values)'] = range_list
    
    file_source_component = [file_match(feature, file_regex) for feature in df.columns]
    column_source_component = [find_original_column(feature, file_regex) for feature in df.columns]
    data_dict_df['source (studies name + column name)'] = [str(file) +": "+ str(column) for file, column in zip(file_source_component, column_source_component)]
    data_dict_df['new column or not (how it was generated)'] = data_dict_df['values (range of the values)'].apply(lambda x: "One-hot Encoding" if x == "0.00 - 1.00" else "Not Encoded")
    data_dict_df['Imputed Or Not'] = data_dict_df['feature name']
    
    
    #Update Column 6 based on Imputed and Not Imputed value
    for index, value in enumerate(data_dict_df['Imputed Or Not']):
        if value in imputed_cols:
            data_dict_df.at[index, 'Imputed Or Not'] = 'Imputed'
        elif value in not_imputed_cols:
            data_dict_df.at[index, 'Imputed Or Not'] = 'Not Imputed'
        else:
            data_dict_df.at[index, 'Imputed Or Not'] = np.nan
            
            
    data_dict_df.to_excel("./TEDDY_data_dictionary.xlsx", index=False)    
    return data_dict_df

In [3]:
data_dict = create_data_dictionary("./TEDDY_data_final.csv", "./Binned_Dataframe.csv", "./Imputed_Labeled_Dataset.csv", file_list_match_regex)
data_dict

,feature name,data type,values (range of the values),source (studies name + column name),new column or not (how it was generated),Imputed Or Not
0,MaskID,int64,200141.00 - 999596.00,None: None,Not Encoded,Not Imputed
1,DIABETES_DIAGNOSIS_BASEEXCESS_negative,int64,0.00 - 1.00,DIABETES_DIAGNOSIS: _BASEEXCESS_negative,One-hot Encoding,NaN
2,DIABETES_DIAGNOSIS_BASEEXCESS_not-reported,int64,0.00 - 1.00,DIABETES_DIAGNOSIS: _BASEEXCESS_not-reported,One-hot Encoding,NaN
3,DIABETES_DIAGNOSIS_BASEEXCESS_positive,int64,0.00 - 1.00,DIABETES_DIAGNOSIS: _BASEEXCESS_positive,One-hot Encoding,NaN
4,DIABETES_DIAGNOSIS_BETAOHB_blood,int64,0.00 - 1.00,DIABETES_DIAGNOSIS: _BETAOHB_blood,One-hot Encoding,NaN
...,...,...,...,...,...,...
47175,teddybook6_12_EVENT_AGE-bin_3_VISIT,float64,-1235778655076855177497542656.00 - 93057051744...,teddybook6_12: VISIT,Not Encoded,Imputed
47176,FAMILY_RELATIVE_binned_GENDER_CD,float64,-109063138217966087419461632.00 - 144833621670...,FAMILY_RELATIVE: GENDER_CD,Not Encoded,Imputed
47177,FAMILY_RELATIVE_binned_HAVE_DIABETES_CD,float64,-1167305418392145810702729216.00 - 15501577719...,FAMILY_RELATIVE: HAVE_DIABETES_CD,Not Encoded,Imputed
47178,FAMILY_RELATIVE_binned_HAVE_OTHER_KIDS_CD,float64,-592263287627394302666080256.00 - 445988245386...,FAMILY_RELATIVE: HAVE_OTHER_KIDS_CD,Not Encoded,Imputed
